<a href="https://colab.research.google.com/github/OsirisValencia/InteligenciaArtificialUdeA/blob/main/03_RandomForestClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '.'
!kaggle
!chmod 600 ./kaggle.json

usage: kaggle [-h] [-v] [-W]
              {competitions,c,datasets,d,kernels,k,models,m,files,f,config}
              ...
kaggle: error: the following arguments are required: command


In [ ]:
!kaggle competitions download -c udea-ai-4-eng-20251-pruebas-saber-pro-colombia
!unzip udea-ai-4-eng-20251-pruebas-saber-pro-colombia.zip

udea-ai-4-eng-20251-pruebas-saber-pro-colombia.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  udea-ai-4-eng-20251-pruebas-saber-pro-colombia.zip
replace submission_example.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: submission_example.csv  
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: test.csv                
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: train.csv               


In [ ]:
import pandas as pd
import numpy as np


dtr = pd.read_csv('train.csv')
dts = pd.read_csv('test.csv')




In [ ]:
# Primeras columnas
df.head(20).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
ID,904256,645256,308367,470353,989032,659872,47159,11829,257869,465511,273010,738026,858669,252472,636397,961327,52868,937873,797253,65292
PERIODO,20212,20212,20203,20195,20212,20203,20183,20183,20212,20183,20203,20203,20183,20195,20195,20203,20203,20203,20183,20212
ESTU_PRGM_ACADEMICO,ENFERMERIA,DERECHO,MERCADEO Y PUBLICIDAD,ADMINISTRACION DE EMPRESAS,PSICOLOGIA,MEDICINA VETERINARIA,INGENIERIA MECANICA,ADMINISTRACIÓN EN SALUD OCUPACIONAL,INGENIERIA INDUSTRIAL,ADMINISTRACION DE EMPRESAS,PSICOLOGIA,ADMINISTRACIÓN FINANCIERA,HOTELERIA Y TURISMO,LICENCIATURA EN CIENCIAS SOCIALES,LICENCIATURA EN PEDAGOGIA INFANTIL,DERECHO,COMUNICACION SOCIAL,ADMINISTRACIÓN EN SALUD OCUPACIONAL,CIENCIA POLITICA,PROFESIONAL EN GESTIÓN DE LA SEGURIDAD Y LA SA...
ESTU_PRGM_DEPARTAMENTO,BOGOTÁ,ATLANTICO,BOGOTÁ,SANTANDER,ANTIOQUIA,ANTIOQUIA,HUILA,BOGOTÁ,ATLANTICO,ANTIOQUIA,SUCRE,BOGOTÁ,BOGOTÁ,CAQUETA,CUNDINAMARCA,BOGOTÁ,BOLIVAR,BOGOTÁ,ANTIOQUIA,BOGOTÁ
ESTU_VALORMATRICULAUNIVERSIDAD,Entre 5.5 millones y menos de 7 millones,Entre 2.5 millones y menos de 4 millones,Entre 2.5 millones y menos de 4 millones,Entre 4 millones y menos de 5.5 millones,Entre 2.5 millones y menos de 4 millones,Más de 7 millones,Entre 2.5 millones y menos de 4 millones,Entre 1 millón y menos de 2.5 millones,Entre 5.5 millones y menos de 7 millones,Entre 2.5 millones y menos de 4 millones,Entre 1 millón y menos de 2.5 millones,Entre 500 mil y menos de 1 millón,Entre 2.5 millones y menos de 4 millones,Menos de 500 mil,Entre 500 mil y menos de 1 millón,Entre 4 millones y menos de 5.5 millones,Entre 4 millones y menos de 5.5 millones,Entre 1 millón y menos de 2.5 millones,Menos de 500 mil,Entre 1 millón y menos de 2.5 millones
ESTU_HORASSEMANATRABAJA,Menos de 10 horas,0,Más de 30 horas,0,Entre 21 y 30 horas,Menos de 10 horas,Entre 21 y 30 horas,Entre 11 y 20 horas,Menos de 10 horas,Más de 30 horas,Entre 11 y 20 horas,Más de 30 horas,Más de 30 horas,0,Entre 11 y 20 horas,Entre 11 y 20 horas,Entre 11 y 20 horas,Más de 30 horas,Más de 30 horas,Menos de 10 horas
FAMI_ESTRATOVIVIENDA,Estrato 3,Estrato 3,Estrato 3,Estrato 4,Estrato 3,Estrato 5,Estrato 2,Estrato 2,Estrato 1,Estrato 5,Estrato 1,Estrato 2,Estrato 3,Estrato 1,Estrato 3,Estrato 3,Estrato 1,Estrato 2,Estrato 1,Estrato 2
FAMI_TIENEINTERNET,Si,No,Si,Si,Si,Si,Si,Si,Si,Si,Si,Si,Si,No,Si,Si,Si,Si,No,Si
FAMI_EDUCACIONPADRE,Técnica o tecnológica incompleta,Técnica o tecnológica completa,Secundaria (Bachillerato) completa,No sabe,Primaria completa,Educación profesional completa,Educación profesional incompleta,Primaria incompleta,Secundaria (Bachillerato) completa,Postgrado,Técnica o tecnológica completa,Educación profesional incompleta,Secundaria (Bachillerato) completa,Técnica o tecnológica completa,Secundaria (Bachillerato) completa,Educación profesional completa,Primaria completa,Secundaria (Bachillerato) completa,Primaria incompleta,Secundaria (Bachillerato) completa
FAMI_TIENELAVADORA,Si,Si,Si,Si,Si,Si,Si,Si,Si,Si,Si,Si,Si,No,Si,Si,No,No,No,Si


In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

# Columnas categorizadas
numeric_cols = ["coef_1", "coef_2", "coef_3", "coef_4"]
ordinal_cols = ["SEMESTRE", "ESTU_HORASSEMANATRABAJA", "FAMI_ESTRATOVIVIENDA", "ESTU_VALORMATRICULAUNIVERSIDAD"]
binary_cols = ["FAMI_TIENEINTERNET", "FAMI_TIENELAVADORA", "FAMI_TIENEAUTOMOVIL", "ESTU_PRIVADO_LIBERTAD", "ESTU_PAGOMATRICULAPROPIO", "FAMI_TIENECOMPUTADOR"]
highcard_cols = ["FAMI_EDUCACIONPADRE", "FAMI_EDUCACIONMADRE"]

facultad_map = {
    +
    'ingenieria': 'Ingeniería y Tecnologías',
    'sistemas': 'Ingeniería y Tecnologías',
    'medicina': 'Ciencias de la Salud y Veterinaria',
    'enfermeria': 'Ciencias de la Salud y Veterinaria',
    'veterinaria': 'Ciencias de la Salud y Veterinaria',
    'administracion': 'Negocios, Economía y Derecho',
    'finanzas': 'Negocios, Economía y Derecho',
    'derecho': 'Negocios, Economía y Derecho',
    'psicologia': 'Ciencias Sociales y Educación',
    'social': 'Ciencias Sociales y Educación',
    'pedagogia': 'Ciencias Sociales y Educación',
    'comunicacion': 'Artes y Comunicación',
    'musica': 'Artes y Comunicación',
    'diseno': 'Artes y Comunicación',
    'biologia': 'Ciencias Naturales y Ambientales',
    'quimica': 'Ciencias Naturales y Ambientales',
    'ambiental': 'Ciencias Naturales y Ambientales',
    'licenciatura': 'Educación y Pedagogía'
}

region_map = {
    **dict.fromkeys([d.lower() for d in ['Antioquia','Cundinamarca','Boyaca','Tolima','Quindio','Risaralda','Santander','Norte de Santander','Huila','Caldas']], 'Andina'),
    **dict.fromkeys([d.lower() for d in ['Atlantico','Bolivar','Magdalena','Cesar','Cordoba','Sucre','La Guajira']], 'Caribe'),
    **dict.fromkeys([d.lower() for d in ['Choco','Valle del Cauca','Cauca','Narino']], 'Pacífica'),
    **dict.fromkeys([d.lower() for d in ['Meta','Arauca','Casanare','Vichada']], 'Orinoquia'),
    **dict.fromkeys([d.lower() for d in ['Amazonas','Caqueta','Guainia','Guaviare','Putumayo','Vaupes']], 'Amazonía'),
    'san andres y providencia': 'Insular'
}

# Mapping for ESTU_HORASSEMANATRABAJA string values to numerical
horas_trabaja_map = {
    'Menos de 10 horas': 0,
    'De 10 a 20 horas': 1,
    'De 21 a 30 horas': 2,
    'Más de 30 horas': 3,
    'No trabaja': -1 # Or some other value indicating no work
}

# Mapping for FAMI_ESTRATOVIVIENDA string values to numerical
estrato_map = {
    'Estrato 1': 1,
    'Estrato 2': 2,
    'Estrato 3': 3,
    'Estrato 4': 4,
    'Estrato 5': 5,
    'Estrato 6': 6,
    'Sin Estrato': 0, # Assign a numerical value for 'Sin Estrato'
    'No aplica': -1 # Assign a numerical value for 'No aplica'
}

# Mapping for binary string values to numerical
binary_map = {
    'Si': 1,
    'No': 0,
    'S': 1,
    'N': 0,

}


def map_facultad(s):
    if pd.isna(s): return 'Otros/Interdisciplinarios'
    s = s.lower()
    for kw, fac in facultad_map.items():
        if kw in s:
            return fac
    return 'Otros/Interdisciplinarios'

def preprocesar(df, imputer_dict=None, encoder=None, high_encoder=None, scaler=None, fit=True):
    df = df.copy()
    if 'RENDIMIENTO_GLOBAL' in df.columns:
        df = df.drop(['ID', 'FAMI_TIENEINTERNET.1', 'RENDIMIENTO_GLOBAL'], axis=1)
    else:
        df = df.drop(['ID', 'FAMI_TIENEINTERNET.1'], axis=1)

    df['PERIODO_STR'] = df['PERIODO'].astype(str).str.zfill(5)
    df['SEMESTRE'] = df['PERIODO_STR'].str[:4].astype(int)
    df = df.drop(['PERIODO', 'PERIODO_STR'], axis=1)

    if fit:
        imputer_dict = {
            'num': SimpleImputer(strategy='median'),
            'ord': SimpleImputer(strategy='most_frequent'),
            'bin': SimpleImputer(strategy='most_frequent'),
            'high': SimpleImputer(strategy='constant', fill_value='Desconocido')
        }
        df[numeric_cols] = imputer_dict['num'].fit_transform(df[numeric_cols])
        # Apply imputation to ordinal columns
        df[ordinal_cols] = imputer_dict['ord'].fit_transform(df[ordinal_cols])
        df[binary_cols] = imputer_dict['bin'].fit_transform(df[binary_cols])
        df[highcard_cols] = imputer_dict['high'].fit_transform(df[highcard_cols])
    else:
        df[numeric_cols] = imputer_dict['num'].transform(df[numeric_cols])
        # Apply imputation to ordinal columns
        df[ordinal_cols] = imputer_dict['ord'].transform(df[ordinal_cols])
        df[binary_cols] = imputer_dict['bin'].transform(df[binary_cols])
        df[highcard_cols] = imputer_dict['high'].transform(df[highcard_cols])

    # Map 'ESTU_HORASSEMANATRABAJA' to numerical values after imputation
    df['ESTU_HORASSEMANATRABAJA'] = df['ESTU_HORASSEMANATRABAJA'].map(horas_trabaja_map).fillna(-2) # Handle potential missing mappings

    # Map 'FAMI_ESTRATOVIVIENDA' to numerical values after imputation
    df['FAMI_ESTRATOVIVIENDA'] = df['FAMI_ESTRATOVIVIENDA'].map(estrato_map).fillna(0) # Handle potential missing mappings, map to 0 or another appropriate value

    # Map 'ESTU_VALORMATRICULAUNIVERSIDAD' to numerical values if it's not already
    # Assuming it might have string values, convert them to numeric.
    # You might need a specific mapping for this column as well if it has non-numeric strings.
    # For now, we'll attempt a direct conversion and handle errors by mapping non-convertible values to a placeholder.
    # Inspect the unique values of this column to create a proper mapping if needed.
    df['ESTU_VALORMATRICULAUNIVERSIDAD'] = pd.to_numeric(df['ESTU_VALORMATRICULAUNIVERSIDAD'], errors='coerce').fillna(-1) # Added line to convert to numeric

    # Map binary columns 'Si'/'No' to 1/0
    for col in binary_cols:
        df[col] = df[col].map(binary_map).fillna(0) # Fill any remaining NaNs or unmapped values with 0


    # One-hot para FACULTAD y REGION
    df['PRGM_NORM'] = df['ESTU_PRGM_ACADEMICO'].astype(str).str.normalize('NFKD').str.encode('ASCII', 'ignore').str.decode('ASCII').str.lower().str.strip()
    df['FACULTAD'] = df['PRGM_NORM'].map(map_facultad)
    df['DEP_NORM'] = df['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.normalize('NFKD').str.encode('ASCII', 'ignore').str.decode('ASCII').str.lower().str.strip()
    df['REGION'] = df['DEP_NORM'].map(region_map).fillna('otros')

    if fit:
        encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
        encoded = encoder.fit_transform(df[['FACULTAD', 'REGION']])
    else:
        encoded = encoder.transform(df[['FACULTAD', 'REGION']])
    encoded_cols = encoder.get_feature_names_out(['FACULTAD', 'REGION'])
    df = pd.concat([df.reset_index(drop=True), pd.DataFrame(encoded, columns=encoded_cols)], axis=1)

    # One-hot para highcard_cols
    if fit:
        high_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
        high_encoded = high_encoder.fit_transform(df[highcard_cols])
    else:
        high_encoded = high_encoder.transform(df[highcard_cols])
    high_encoded_cols = high_encoder.get_feature_names_out(highcard_cols)
    df = pd.concat([df.reset_index(drop=True), pd.DataFrame(high_encoded, columns=high_encoded_cols)], axis=1)

    # Normalización final (opcional pero recomendable para algunos modelos)
    # scale_cols remains the same, but ordinal_cols should now contain only numerical data
    scale_cols = numeric_cols + ordinal_cols # ordinal_cols now includes numerical representations
    # Add binary_cols to scale_cols since they are now numerical
    scale_cols.extend(binary_cols)
    if fit:
        scaler = StandardScaler()
        df[scale_cols] = scaler.fit_transform(df[scale_cols])
    else:
        df[scale_cols] = scaler.transform(df[scale_cols])

    # Elimina columnas innecesarias
    df = df.drop(['ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO', 'PRGM_NORM',
                  'DEP_NORM', 'FACULTAD', 'REGION'] + highcard_cols, axis=1)

    return df, imputer_dict, encoder, high_encoder, scaler

In [ ]:

dtr.shape, dts.shape

((692500, 21), (296786, 20))

In [ ]:
source_cols = [i for i in dtr.columns if i!="RENDIMIENTO_GLOBAL"]
all_data_concat = pd.concat((dtr[source_cols], dts[source_cols]))
all_data_concat.index = range(len(all_data_concat))

# Unpack all five returned values from the preprocesar function
processed_all_data, imputer_dict, encoder, high_encoder, scaler = preprocesar(all_data_concat)

# Define lentr as the length of the original training data
lentr = len(dtr)
lents = len(dts)

# Use the processed_all_data DataFrame for slicing
Xtrk, ytrk = processed_all_data.iloc[:lentr].values, dtr["RENDIMIENTO_GLOBAL"].values
Xtsk  = processed_all_data.iloc[lentr:].values # Corrected slicing for Xtsk

print (Xtrk.shape, ytrk.shape)

(692500, 54) (692500,)


In [ ]:
n = int(len(Xtrk)*0.5)
n

346250

In [ ]:
idxs = np.random.permutation(len(Xtrk))
idxs_trm = idxs[:n]
idxs_tsm = idxs[n:]

Xtrm = Xtrk[idxs_trm]
ytrm = ytrk[idxs_trm]

Xtsm = Xtrk[idxs_tsm]
ytsm = ytrk[idxs_tsm]

print (Xtrm.shape, ytrm.shape, Xtsm.shape, ytsm.shape)

(346250, 54) (346250,) (346250, 54) (346250,)


In [ ]:

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Split training data for validation
X_train, X_val, y_train, y_val = train_test_split(Xtrk, ytrk, test_size=0.2, random_state=42)

# Initialize and train a RandomForestClassifier model
# RandomForest is a good choice for classification and handles various data types well.
model = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=10)
model.fit(X_train, y_train)

# Train the model on the training data
model.fit(X_train, y_train)

# Make predictions on the validation set
y_pred = model.predict(X_val)

# Evaluate the model
accuracy = accuracy_score(y_val, y_pred)
report = classification_report(y_val, y_pred)

print(f"Accuracy on validation set: {accuracy:.4f}")
print("\nClassification Report on validation set:")
report

# Now, you can train the model on the entire training data (Xtrk, ytrk)
# and make predictions on the test data (Xtsk) for submission if needed.
# model.fit(Xtrk, ytrk)
# test_predictions = model.predict(Xtsk)

# If you need to save the test predictions, you would typically create a submission file
# submission_df = pd.DataFrame({'ID': dts['ID'], 'RENDIMIENTO_GLOBAL': test_predictions})
# submission_df.to_csv('submission.csv', index=False)



Accuracy on validation set: 0.3811

Classification Report on validation set:


'              precision    recall  f1-score   support\n\n        alto       0.44      0.60      0.51     35165\n        bajo       0.39      0.55      0.46     34573\n  medio-alto       0.29      0.17      0.21     34259\n  medio-bajo       0.31      0.21      0.25     34503\n\n    accuracy                           0.38    138500\n   macro avg       0.36      0.38      0.36    138500\nweighted avg       0.36      0.38      0.36    138500\n'

In [ ]:
model.fit(Xtrk, ytrk)

# Make predictions on the test data (Xtsk)
test_predictions = model.predict(Xtsk)

# Create the submission DataFrame using the IDs from the original test DataFrame (dts)
# and the predictions made on the actual test data (Xtsk)
submission_df = pd.DataFrame({'ID': dts['ID'], 'RENDIMIENTO_GLOBAL': test_predictions})

# Save the submission file
submission_df.to_csv('submission.csv', index=False)


In [ ]:
!kaggle competitions submit -c udea-ai-4-eng-20251-pruebas-saber-pro-colombia -f submission.csv -m "Message"

100% 3.90M/3.90M [00:01<00:00, 2.18MB/s]
Successfully submitted to UDEA/ai4eng 20251 - Pruebas Saber Pro Colombia